<a href="https://colab.research.google.com/github/fbeilstein/dbms/blob/master/DB_lecture_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Paxos**

Messages reach their destination eventually, at an unknown time. If we need to deliver messages in order, we have to use the **atomic broadcast** (also called the **total order multicast**), which guarantees both reliable delivery and total order.


An **atomic broadcast** is a problem equivalent to **consensus** in an asynchronous system with crash failures, since participants have to agree on the message order and must be able to learn about it. You will see many similarities in both motivation and implementation between atomic broadcast and consensus algorithms.


Probably the most widely known consensus algorithm is **Paxos**. It was first introduced by Leslie Lamport in [“The Part-Time Parliament” paper](https://lamport.azurewebsites.net/pubs/lamport-paxos.pdf). In 2001, the author released a follow-up paper titled [“Paxos Made Simple”](https://lamport.azurewebsites.net/pubs/paxos-simple.pdf) that introduced simpler terms, which are now commonly used to explain this algorithm.


Participants in Paxos can take one of three roles: 
* **Proposers** Receive values from clients, create proposals to accept these values, and attempt to collect votes from acceptors.
* **Acceptors** Vote to accept or reject the values proposed by the proposer. For fault tolerance, the algorithm requires the presence of multiple acceptors, but for liveness, only a quorum (majority) of acceptor votes is required to accept the proposal.
* **Learners** Take the role of replicas, storing the outcomes of the accepted proposals. 

Any participant can take any role, and most implementations colocate them: a single process can **simultaneously** be a proposer, an acceptor, and a learner.

Every proposal consists of a value, proposed by the client, and a unique monotonically increasing proposal number. This number is then used to ensure a total order of executed operations and establish happened-before/after relationships among them.

* **consensus** is agreeing on one result
* once **majority** agrees this is a consensus
* consensus will be **eventually** known by **everyone**
* involved parties want to agree on **any** result
* messages can get lost

**Paxos Algorithm**


The Paxos algorithm can be generally split into two phases: 
* voting (or propose phase)
* replication. 

During the **voting** phase, proposers compete to establish their leadership. 

During **replication**, the proposer distributes the value to the acceptors.
The proposer is an initial point of contact for the client. It receives a value that should be decided upon, and attempts to collect votes from the quorum of acceptors. When this is done, acceptors distribute the information about the agreed value to the learners, ratifying the result. Learners increase the replication factor of the value that’s been agreed on.


**Only one** proposer can collect the majority of votes. Under some circumstances, votes may get split evenly between the proposers, and neither one of them will be able to collect a majority during this round, forcing them to restart. 

During the propose phase, the proposer sends a Prepare(n) message (where n is a
proposal number) to a majority of acceptors and attempts to collect their votes.
When the acceptor receives the prepare request, it has to respond, preserving the following invariants:

**propose phase**

proposer -"prepare", message # $N$-> acceptors (majority of)

Message from proposer|Acceptor state ->|Responce| -> Acceptor state|
---|---|---|---|
prepare $N$|not prepared| promise to accept $N$ | prepared for $N$
prepare $N$|prepared for $M < N$| promise to accept $N$ | prepared for $N$
prepare $N$|prepared for $M > N$| promise to accept $M$ | prepared for $M > N$
prepare $N$|accepted $M$, value $v$ | notify that $M$ with value $v$ accepted | accepted $M$, value $v$ 
**proposer chooses as $v$ the value with highest-numbered promise (may be its own)**
accept $N$, $v$ | prepared or accepted $M \leq N$ | notify proposer and readers | accepted $N$, value $v$
accept $N$, $v$ | prepared or accepted $M > N$ | ignore | prepared or accepted $M > N$

* Acceptor can respond to more than one prepare request, as long as the later one has a higher sequence number.
* **accepted** state at any node means there was a consensus at some point, we might want to just disseminate the accepted value.

**replication phase**



During the replication phase, after collecting a majority of votes, the proposer can start the replication, where it commits the proposal by sending acceptors an Accept message with value v and proposal number n. v is the value associated with the highest-numbered proposal among the responses it received from acceptors, or any value of its own if their responses did not contain old accepted proposals.

Once a consensus was reached on the value (in other words, it was accepted by at
least one acceptor), future proposers have to decide on the same value to guarantee the agreement. This is why acceptors respond with the latest value they’ve accepted. If no acceptor has seen a previous value, the proposer is free to choose its own value.


A learner has to find out the value that has been decided, which it can know after receiving notification from the majority of acceptors. To let the learner know about the new value as soon as possible, acceptors can notify it about the value as soon as they accept it. If there’s more than one learner, each acceptor will have to notify each learner. One or more learners can be distinguished, in which case it will notify other learners about accepted values.

**Quorums in Paxos**


A **quorum** is the minimum number of votes required for the operation to be performed. This number usually constitutes a **majority** of participants. Quorums are used to make sure that some of the participants can fail, but we still can proceed as long as we can collect votes from the alive ones.


The main idea behind quorums is that even if participants fail or happen to be separated by the network partition, there’s at least one participant that acts as an arbiter, ensuring protocol correctness. Once a sufficient number of participants accept the proposal, the value is guaranteed to be accepted by the protocol, since any two majorities have at least one participant in common (**pigeonhole principle**).


To guarantee **safety**, for each step we have to wait for responses from **at least** a quorum of nodes. We can send proposals and accept commands to more nodes; we just do not have to wait for their responses to proceed. There’s no configuration that can produce incorrect or inconsistent states since this would contradict the definition of consensus.


To guarantee **liveness**, we can proceed as soon as we hear from the quorum. Liveness is guaranteed in the presence of $f$ failed processes. For that, the protocol requires $2f + 1$ processes in total so that, if f processes happen to fail, there are still f + 1 processes able to proceed.

**Failure Scenarios**


**Failure scenario:** proposer fails on voting -> we can just ignore,will be overwritten later

**Failure scenario:** proposer fails during the second phase, before it is able to broadcast the value to all the acceptors -> new proposer may pick up and commit the value, distributing it to the other participants.


proposer 1 | proposer 2 | acceptor 1 | acceptor 2 | acceptor 3
---|---|---|---|---
propose id 1, value 'X' | idle | accept 1 | accept 1 | accept 1 
commit value 'X',id 1 | idle | accepted value 'X' id 1 | accept 1 | accept 1 
FAIL | idle | accepted value 'X' id 1 | accept 1 | accept 1
FAIL | propose id 2, value 'Y' | notify on accepted value 'X' id 1 | accept 2 | accept 2
FAIL | already was quorum on 'X' !
FAIL | commit value 'X' id 2 | accepted value 'X' id 2 | accepted value 'X' id 2 | accepted value 'X' id 2

**Note:** all of it may happen without the original proposer knowing anything.
In a client/server application, where the client is connected only to the original proposer, this might lead to situations where the client doesn’t know about the result of the Paxos round execution.

**Failure scenario:** proposer fails during the second phase, before it is able to broadcast the value to all the acceptors -> new proposer may pick up and commit **new** value, distributing it to the other participants.

proposer 1 | proposer 2 | acceptor 1 | acceptor 2 | acceptor 3
---|---|---|---|---
propose id 1, value 'X' | idle | accept 1 | accept 1 | accept 1 
commit value 'X',id 1 | idle | accepted value 'X' id 1 | accept 1 | accept 1 
FAIL | idle | accepted value 'X' id 1 | accept 1 | accept 1
FAIL | propose id 2, value 'Y' | message lost! | accept 2 | accept 2
FAIL | there is still quorum !
FAIL | commit value 'Y' id 2 | accepted value 'Y' id 2 | accepted value 'Y' id 2 | accepted value 'Y' id 2

**Failure scenario:** proposer **and** acceptor fail during the second phase -> commited values may diverge.


proposer 1 | proposer 2 | acceptor 1 | acceptor 2 | acceptor 3
---|---|---|---|---
propose id 1, value 'X' | idle | accept 1 | accept 1 | accept 1 
commit value 'X',id 1 | idle | accepted value 'X' id 1 | accept 1 | accept 1 
FAIL | idle | accepted value 'X' id 1 | accept 1 | accept 1
FAIL | idle | FAIL | accept 1 | accept 1
FAIL | propose id 2, value 'Y' | FAIL | accept 2 | accept 2
FAIL | there is still quorum !
FAIL | commit value 'Y' id 2 | FAIL | accepted value 'Y' id 2 | accepted value 'Y' id 2
FAIL | idle | UP, accepted value 'X' id 1 | accepted value 'Y' id 2 | accepted value 'Y' id 2

**Note:** Any proposer that comes after this round that will overlap with A1, will ignore A1’s value and choose a more recent accepted proposal instead.

**Failure scenario:** two or more proposers start competing, each trying
to get through the propose phase, but keep failing to collect a majority because the other one beat them to it.

proposer 1 | proposer 2 | acceptor 1 | acceptor 2 | acceptor 3
---|---|---|---|---
propose id 1, value 'X' | idle | accept 1 | accept 1 | accept 1 
ready to commit | propose id 2, value 'Y' | accept 2 | accept 2 | accept 2 
commit value 'X', id 1 | ready to commit | notify on id 2 | notify on id 2 | notify on id 2
propose id 3, value 'X' | ready to commit | accept 3 | accept 3 | accept 3
ready to commit | commit value 'Y', id 2 | notify on id 3 | notify on id 3 | notify on id 3 
ready to commit | propose id 4, value 'Y' | accept 4 | accept 4 | accept 4


**Note:** This problem is usually solved by incorporating a random backoff, which eventually lets one of the proposers proceed while the other one
sleeps. 

The Paxos algorithm can tolerate acceptor failures, but only if there are still enough acceptors alive to form a majority

**Multi-Paxos**

* classic Paxos = proposition + acceptance

We can think of single-decree Paxos as a write-once register: we have a slot where we can put a value, and as soon as we’ve written the value there, no subsequent modifications are possible. During the first step, proposers compete for ownership of the register, and during the second phase, one of them writes the value. 

* Multi-Paxos = many propositions and acceptances

At the same time, Multi-Paxos can be thought of as an append-only log, consisting of a sequence of such values: we can write one value at a time, all values are strictly ordered, and we cannot modify already written values.


**why proposition phase?** 

This is basically the way to establish an **arbitrary** proposer. Only after the proposer is established for the round, which happens after a **majority** of acceptors respond with a Promise to the proposer’s Prepare, can it start the replication. 

(-) It is wasteful that a propose round is required for each replication round that occurs in the system.  

**what if majority knows proposer?** 

* Having an established leader (**distinguished proposer**) in **Multi-Paxos**, we can skip the propose phase and proceed straight to replication.

Some Multi-Paxos implementations use **leases**.  The leader periodically contacts the participants, notifying them that it is still alive, effectively prolonging its lease. Participants have to respond and allow the leader to continue operation, promising that they will not accept proposals from other leaders for the period of the lease.

* Instead of proposing every round, we get a lease and commit for few rounds.

**Leases** are not a correctness guarantee, but a performance optimization that allows reads from the active leader without collecting a quorum. To guarantee safety, **leases rely on the bounded clock synchrony** between the participants. 

Proposer 2 = distinguished, lease active. note that requests go through distinguished proposer.

proposer 1 | *proposer 2 | acceptor 1 | acceptor 2 | acceptor 3
---|---|---|---|---
idle | commit value 'X' | accepted value 'X' | accepted value 'X' | accepted value 'X' 
notify on value 'Y' -> | value 'Y' proposed | idle | idle | idle 
idle | commit value 'Y' | accepted value 'Y' | accepted value 'Y' | accepted value 'Y'


**Problem:** If their clocks drift too much and the leader assumes its lease is still valid while other participants think its lease has expired, linearizability cannot be guaranteed.


To preserve the state in case of process crashes, participants keep a durable log of received messages. To prevent a log from growing indefinitely large it is periodically synchronized with a primary structure, creating a snapshot, the log can be truncated. Log and state snapshots should be mutually consistent, and snapshot changes should be applied atomically with truncation of the log segment.

**Fast Paxos**


We can reduce the number of round-trips by one, compared to the classic Paxos algorithm, by letting any proposer contact acceptors directly rather than going through the leader. This optimization is called [**Fast Paxos**](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr-2005-112.pdf). Some Fast Paxos descriptions say that clients can contact acceptors directly.


The classic Paxos algorithm has a condition, where during the replication phase, the proposer can pick any value it has collected during the propose phase. Fast Paxos has two types of rounds: 
* classic, where the algorithm proceeds the same way as the classic
version, 
* fast, where it allows acceptors to accept other values.

**fast round**

proposer 1 | *proposer 2 | acceptor 1 | acceptor 2 | acceptor 3
---|---|---|---|---
idle | accept any | ready for accept | ready for accept | ready for accept 
commit 'Y'| idle | accept 'Y', notify 2 | accept 'Y', notify 2 | accept 'Y', notify 2

**Proposer** that has collected a sufficient number of responses during the propose phase = **coordinator**.


This algorithm is **prone to collisions**, which occur if two or more proposers attempt to use the fast step and reduce the number of round-trips, and acceptors receive different values. The coordinator has to intervene and start recovery by initiating a new round.

For this, we need to increase the quorum size to $2f + 1$ (where $f$ is the number of processes allowed to fail), compared to $f + 1$ in classic Paxos, and a total number of acceptors to $3f + 1$.


One of the disadvantages of Fast Paxos is the increased number of round-trips and request latency on collisions if the request rate is high.

**Egalitarian Paxos**


single leader = point of failure + disproportional load

* Many systems split the range of possible values into smaller segments and allow a part of the system to be responsible for a specific range without having to worry about the other parts (**partitioning**).
* Use a leader responsible for the commit of the specific command, and establish the order by looking up and setting dependencies. This approach is commonly called [**Egalitarian Paxos**](https://www.pdl.cmu.edu/PDL-FTP/associated/CMU-PDL-13-111.pdf), or **EPaxos**. 


EPaxos starts with a **Pre-Accept** phase, during which a process becomes a leader for the specific proposal. Every proposal has to include:
* **Dependencies** All commands that potentially interfere with a current proposal, but are not necessarily already committed.
* **A sequence number**
This breaks cycles between the dependencies. Set it with a value larger than any
sequence number of the known dependencies.


**(1)** After collecting this information, leader forwards a **Pre-Accept** message to a fast quorum of replicas. A fast quorum is ⌈3f/4⌉ replicas, where f is the number of tolerated failures.

**(2)** Replicas check their local command logs, update the proposal dependencies based on their view of potentially conflicting proposals, and send this information back to the leader. 

**(3a)** If the leader receives responses from a fast quorum of replicas, and their dependency lists are in agreement with each other and the leader itself, it can commit the command.

**(3b)** If the leader does not receive enough responses or if the command lists received from the replicas differ and contain interfering commands, it updates its proposal with a new dependency list and a sequence number. 

The new dependency list is based on previous replica responses and combines all collected dependencies. The new sequence number has to be larger than the highest sequence number seen by the replicas. After that, the leader sends the new, updated command to ⌊f/2⌋ + 1 replicas.

**(4)** After this is done, the leader can finally commit the proposal. 


Effectively, we have two possible scenarios:
* **Fast path** When dependencies match and the leader can safely proceed with the commit phase with only a fast quorum of replicas.
* **Slow path** When there’s a disagreement between the replicas, and their command lists have to be updated before the leader can proceed with a commit.

**Note:** it is not a problem to share data between acceptors, it is a problem to apply operations in one order without messing up local storage.

proposer 1 | proposer 2 | acceptor 1 | acceptor 2 | acceptor 3
---|---|---|---|---
pre accept (1, 'X', {}) | idle | [], idle | [], idle | [], idle 
idle | idle | [], accept 'X', notify 1 | [], accept 'X', notify 1 | [], idle
commit 'X' | pre accept (2, 'Y', {}) | ['X'], idle | ['X'], notify 2 on conflict | [], accept 'Y', notify 2
idle | pre accept (2, 'Y', {1}) | ['X'], idle | ['X'], accept 'Y', notify 2 | req other for id 1
idle | collectig responces | ['X'], idle | ['X'], accept 'Y'| ['X'], accept 'Y', notify 2
idle | commit | ['X'], idle | ['X','Y'], idle | ['X','Y'], idle


Commit is done by responding to the client and asynchronously notifying replicas
with a Commit message. Commands are executed after they’re committed.

Since dependencies are collected during the Pre-Accept phase, by the time requests are executed, the command order is already established and no command can suddenly appear somewhere in-between: it can only get appended after the command with the largest sequence number.

To execute a command, replicas build a dependency graph and execute all commands
in a reverse dependency order. In other words, before a command can be executed,
all its dependencies (and, subsequently, all their dependencies) have to be executed.

Since only interfering commands have to depend on each other, this situation should be relatively rare for most workloads.

Similar to Paxos, EPaxos uses proposal numbers, which prevent stale messages from being propagated. Sequence numbers consist of an epoch (identifier of the current cluster configuration that changes when nodes leave and join the cluster), a monotonically incremented node-local counter, and a replica ID. 

If a replica receives a proposal with a sequence number lower than one it has already seen, it negatively acknowledges the proposal, and sends the highest sequence number and an updated command list known to it in response.

**Flexible Paxos**


A quorum is usually defined as a majority of processes. By definition, we have an intersection between two quorums no matter how we pick nodes: there’s always at least one node that can break ties.


We have to answer two important questions:
* Is it necessary to contact the majority of servers during every execution step?
* Do all quorums have to intersect? In other words, does a quorum we use to pick
a distinguished proposer (first phase), a quorum we use to decide on a value (second phase), and every execution instance (for example, if multiple instances of the second step are executed concurrently), have to have nodes in common?


Since we’re still talking about consensus, we cannot change any safety definitions: the algorithm has to guarantee the agreement.

It turns out in Paxos we only require the group of nodes from the first phase (that elects a leader) to overlap with the group from the second phase (that participates in accepting proposals). In other words, a **quorum** doesn’t have to be defined as a **majority**, but only as a **nonempty group of nodes**. 

If we define a total number of participants as N, the number of nodes required for a propose phase to succeed as $P$, and the number of nodes required for the accept phase to succeed as $A$, we only need to ensure that $A+P>N$. Since the second phase is usually more common than the first one, $A$ can contain
much less acceptors than $P$. 

This finding is an important observation crucial for understanding consensus. The algorithm that uses this approach is called [**Flexible Paxos**](https://arxiv.org/pdf/1608.06696.pdf).

For example, if we have five acceptors, as long as we require collecting votes from four of them to win the election round, we can allow the leader to wait for responses from two nodes during the replication stage. Moreover, since there’s an overlap between any subset consisting of two acceptors with the leader election quorum, we can submit proposals to disjoint sets of acceptors. Intuitively, this works because whenever a new leader is elected without the current one being aware of it, there will always be at least one acceptor that knows about the existence of the new leader.

**Flexible Paxos** allows trading availability for latency: we reduce the number of nodes participating in the second phase but have to collect more votes, requiring more participants to be available during the leader election phase. 

Another Paxos variant using the idea of intersecting quorums is **Vertical Paxos**. Vertical Paxos distinguishes between read and write quorums. These quorums must intersect. A leader has to collect a smaller read quorum for one or more lower-numbered proposals, and a larger write quorum for its own proposal.

**Generalized Solution to Consensus**



Paxos might sometimes be a bit difficult to reason about: multiple roles, steps, and all the possible variations are hard to keep track of. But we can think of it in simpler terms. Instead of splitting roles between the participants and having decision rounds, we can use a simple set of concepts and rules to achieve guarantees of a single-decree Paxos. We discuss this approach only briefly as this is a [relatively new development](https://arxiv.org/pdf/1902.06776.pdf) —it’s important to know, but we’ve yet to see its implementations and practical applications.


We have a **client** and a set of **servers**. Each server has multiple **registers**. A register has an **index** identifying it, can be **written only once**, and it can be in one of three states:
* unwritten
* containing a value
* containing **nil** (a special empty value)

Registers with the **same** index located on different servers form a **register set**. Each register set can have one or more quorums. Depending on the state of the registers in it, a quorum can be in one of the states:
* **Any** Depending on future operations, this quorum set can decide on any value.
* **Maybe v** If this quorum reaches a decision, its decision can only be v.
* **None** This quorum cannot decide on the value.
* **Decided v** This quorum has decided on the value v.

The client exchanges messages with the servers and maintains a state table, where it keeps track of values and registers, and can infer decisions made by the quorums.

To maintain correctness, we have to limit how clients can interact with servers and which values they may write and which they may not. 

* Client can output the decided value only if it has read it from the quorum of servers in the same register set.
* It is allowed to write a specific value to the register only if it has received it as input or has read it from a register (no out-of-nowhere creation).  
* Clients cannot write values that allow different quorums in the same register to decide on different values. 
* Clients cannot write values that override previous decisions made in the previous register sets (decisions made in register sets up to r - 1 have to be None, Maybe v, or Decided v)

**Generalized Paxos algorithm**


* **phase 1**
The client checks if the register it is about to write is unwritten by sending a
P1A(*register*) command to the server. If the register is unwritten, all registers up to *register* are set to nil, which prevents clients from writing to previous registers (message id # in paxos). The server responds with a set of registers written so far. If it receives responses from the majority of servers, the client chooses either the nonempty value from the register with the largest index or its own value in case no value is present (paxos = found commited value). Otherwise, it restarts the first phase.
* **phase 2**
The client notifies all servers about the value it has picked during the first phase by sending them P2A(register, value) (paxos commit). If the majority of servers respond to this message, it can output the decision value. Otherwise, it starts again from phase 1.

